# Projet KAYAK

In [2]:
#importing libraries to scrape data
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

In [3]:
#list of cities in France to recommand 
cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg",
            "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
            "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes",
            "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban",
            "Biarritz", "Bayonne", "La Rochelle"]

### Get the gps coordinates and weather forecast of all the cities with api query 

In [4]:
import requests
from datetime import datetime

In [5]:
# example Paris , France
paris_gps = requests.get("https://nominatim.openstreetmap.org/search?q=Paris+France&format=geojson").json()['features'][0]['geometry']['coordinates']
display(paris_gps)

[2.3200410217200766, 48.8588897]

In [6]:
# getting weather info from api.openweathermap
key = "274c0a55230e8d4d1cb650d6dfd9b351"
r = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude=current,minutely,hourly,alerts&units=metric&appid={}".format(paris_gps[1],paris_gps[0], key)).json()

In [7]:
#parmaters chosen to calcualte average forcasted weather for each city over the next 7 days
weather_params = ['sunrise','sunset','temp','humidity']

#create a funtction that request from openweathermap weather forecast data for each city in France given it's gps position
def weather_forecast(city):
    key = "274c0a55230e8d4d1cb650d6dfd9b351"
    city_gps = requests.get("https://nominatim.openstreetmap.org/search?q={}+France&format=geojson".format(city)).json()['features'][0]['geometry']['coordinates']
    df = pd.DataFrame(columns = ['temp','humidity'])
    r = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude=current,minutely,hourly,alerts&units=metric&appid={}".format(city_gps[1],city_gps[0], key)).json()
    for i in range(8):
        r['daily'][i] # get forcast of day i
        newDict = {key: value for (key, value) in r['daily'][i].items() if key in weather_params} #keeping only weather param values needed
        #adding a new value : duration of sunlight in the day as number of hours
        newDict['sunlight duration']=round((newDict['sunset']-newDict['sunrise'])/3600,2)
        newDict['temp'] = newDict['temp']['day'] #we choose to keep the avg temperature among all the temp data
        del newDict['sunrise']
        del newDict['sunset']
        df.loc[i,'temp'] = newDict['temp']
        df.loc[i,'humidity'] = newDict['humidity']
        df.loc[i,'sunlight duration'] = newDict['sunlight duration']
    #print("weather forecast in : {}".format(city))
    return {'avg day temp' :round(df['temp'].mean(),2), \
            'avg humidity' : round(df['humidity'].mean(),2), \
            'avg sunlight in hours' : round(df['sunlight duration'].mean(),2)}

weather_forecast('Mont Saint Michel')

{'avg day temp': 9.9, 'avg humidity': 66.75, 'avg sunlight in hours': 11.27}

In [ ]:
#getting the data for all french cities with a loop and putting it into a dataframe 

In [9]:
cities_weather = pd.DataFrame(columns = ['id','city name', 'avg day temp','avg humidity', 'avg sunlight in hours'])
for i in range(len(cities)):
    cities_weather.loc[i,'city name'] = cities[i]
    cities_weather.loc[i,'avg day temp'] = weather_forecast(cities[i])['avg day temp']
    cities_weather.loc[i,'avg humidity'] = weather_forecast(cities[i])['avg humidity']
    cities_weather.loc[i,'avg sunlight in hours'] = weather_forecast(cities[i])['avg sunlight in hours']
    cities_weather.loc[i,'id']=i

In [10]:
cities_weather.head()

,id,city name,avg day temp,avg humidity,avg sunlight in hours
0,0,Mont Saint Michel,9.9,66.75,11.27
1,1,St Malo,9.15,67.88,11.27
2,2,Bayeux,9.35,64.5,11.26
3,3,Le Havre,8.89,62.0,11.25
4,4,Rouen,9.28,54.62,11.25


In [11]:
cities_weather.to_csv('weather data.csv')

In [13]:
#Use plotly to display the best destinations on a map

### Get the hotel info of all the cities with booking scarpping

In [3]:
#installing scrappy
!pip install Scrapy

In [1]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class BookingSpider(scrapy.Spider):
    # Name of your spider
    name = "booking"

    # Starting URL
    start_urls = ['https://www.booking.com/city/fr/paris.fr.html']

    # Parse function for form request
    #def parse(self, response):
    #    # FormRequest used to make a search in Paris
    #    return scrapy.FormRequest.from_response(
    #        response,
    #        formdata={'find_loc': 'paris'},
    #        callback=self.after_search
    #    )

    # Callback used after login
    def after_search(self, response):
        
        results = response.css('.sr__card')
        
        for r in results:
            yield {
                'name': r.css('span.bui-card__title::text').get(),
               # 'url': 'https://www.booking.com/hôtels' + r.attrib["href"]
               # 'description' : 
            }

In [3]:
# Name of the file where the results will be saved
filename = "hotels-paris.json"

# If file already exists, delete it before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir('booking/'):
        os.remove('booking/' + filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36', #changer user 
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'booking/' + filename : {"format": "json"},
    },
    "AUTOTHROTTLE_ENABLED": True  # AutoThrottle Here!
})

# Start the crawling using the spider you defined above
process.crawl(BookingSpider)
process.start()

2022-03-03 16:57:29 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-03-03 16:57:29 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) - [GCC 9.4.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 36.0.1, Platform Linux-5.4.144+-x86_64-with-glibc2.31
2022-03-03 16:57:29 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}
2022-03-03 16:57:29 [scrapy.extensions.telnet] INFO: Telnet Password: fc0e941eecd62457
2022-03-03 16:57:29 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.f

In [ ]:
class BookingSpider(scrapy.Spider): #ma classe herite de la class scrapy.Spider? càd elle n'est pas vierge 
    # Name of your spider
    name = "paris" #attribut name de la classe Spider 

    # Url to start your spider from #attribut name de la classe Spider
    start_urls = [
        'https://www.booking.com/searchresults.fr.html?aid=304142&label=gen173nr-1DCBcoggI46AdIM1gEabooking%20paris2IAQGYAQ24ARfIAQzYAQPoAQGIAgGoAgO4AqDHg5EGwAIB0gIkMGM4OTVmMDktYTdkNy00M2Q1LWEwMmMtNDQ3Y2I1ZGNlOTgx2AIE4AIB&sid=5be3da18843cb0373296f31b70eae6c5&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D304142%3Blabel%3Dgen173nr-1DCBcoggI46AdIM1gEaE2IAQGYAQ24ARfIAQzYAQPoAQGIAgGoAgO4AqDHg5EGwAIB0gIkMGM4OTVmMDktYTdkNy00M2Q1LWEwMmMtNDQ3Y2I1ZGNlOTgx2AIE4AIB%3Bsid%3D5be3da18843cb0373296f31b70eae6c5%3Btmpl%3Dsearchresults%3Bac_click_type%3Db%3Bac_position%3D0%3Bcity%3D-2592467%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Biata%3DPAR%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D31d76fa92eb10499%3Bss%3DParis%252C%2520%25C3%258Ele-de-France%252C%2520France%3Bss_all%3D0%3Bss_raw%3Dparis%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DChipping%2520Norton%3Bssne_untouched%3DChipping%2520Norton%26%3B&ss=Paris&is_ski_area=0&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&from_sf=1',
    ]

    # Callback function that will be called when starting your spider
    # It will get text, author and tags of the first <div> with class="quote"; 
    def parse(self, response):  #response est un argument de la méthode parse qui correspond à l'objet soup de BeautifullSoup
        quote = response.css('div.datatest-id')
        data = { #ma data est rangée expres en dictionnaire qui sera retourné par la method Parse
            'name': quote.css('span.text::text').get(),
            'description': quote.css('span small.author::text').get(),
            'url': quote.css('div.tags a.tag::text').getall(),
            'score': quote.css('span small.author::text').get(),
            'coordinates': quote.css('span small.author::text').get(),
        }
        return data

In [ ]:
//*[@id="search_results_table"]/div/div/div/div/div[6]/div[3]/div[1]/div[1]/div/a

In [ ]:
//*[@id="c-lp-top-hotels"]/div[2]/div[4]/div[1] #xpath de la liste des hotels 